<br>
<a id='intro'></a>
<br>
<br>
<br>

# Tools for ICOS data availability notebook
This notebook contains all the functions used in the ```icos_data_availability_table.ipynb``` notebook.
The notebook is divided in the following parts:

- [Import tools](#import_tools)
- [Control functions](#control_funcs)
- [Time functions](#time_funcs)
- [Station name function](#st_name_funcs)
- [Data availability functions](#availability_funcs)
- [Update availability plot function](#update_availability_table_func)
- [Widget function](#widget_func)


Use the links to swiftly transfer to different parts of the notebook.

<br>
<br>

<a id='import_tools'></a>

## 1. Import tools

In [ ]:
#Import modules:
import pandas as pd 

#Import ICOS tools:
from icoscp.station import station
from icoscp.cpb.dobj import Dobj
from icoscp.sparql.runsparql import RunSparql
from icoscp.sparql import sparqls

import warnings
#warnings.filterwarnings('ignore')

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='control_funcs'></a>

## 2. Control functions

In [ ]:
#Function that checks if a variable value can be 
#converted to float:
def isfloat(value):
    
    try:
        float(value)
        return True
    
    except ValueError:
        return False


<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='time_funcs'></a>

## 3. Time functions

In [ ]:
def toYearFraction(date):
    
    #Import module:
    from datetime import datetime
    
    def sinceEpoch(date): # returns seconds since epoch
        
        #Import module:
        import time 
        
        return time.mktime(date.timetuple())
    
    s = sinceEpoch

    year = date.year
    startOfThisYear = datetime(year=year, month=1, day=1)
    startOfNextYear = datetime(year=year+1, month=1, day=1)

    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    fraction = yearElapsed/yearDuration

    return date.year + fraction

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='st_name_funcs'></a>

## 4. Station name function

In [ ]:
#Function that takes a row of a dataframe as input and
#returns a string with the combined strings of
#icos station ID and sampling height (e.g. 'GAT030').
def st_str(row, col1, col2):
    
    if (row[col1] is None):
        
        #Store sampling height that is ''
        sh_val = ''
        
    else:
        
        if(isfloat(row[col1])):
    
            #Add a zero infront of sampling height values that are <100:
            if (float(row[col1])<100) & (float(row[col1])>=10):

                #Store sampling height as string (>=10 & <100):
                sh_val = '0'+row[col1]

            #Add 2 zeros infront of sampling height values that are <10:
            elif (float(row[col1])<10) & (float(row[col1])>0):

                #Store sampling height as string (<10):
                sh_val = '00'+row[col1]

            else:
                #Store sampling height as string (>=100)
                sh_val = row[col1]

        else:
            #Store sampling height that is ''
            sh_val = ''

    
    #Return stationId as combination of code and sampl. height (e.g. 'GAT030'):
    return row[col2]+sh_val.replace('.0', '')

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='availability_funcs'></a>

## 5. Availability functions

In [ ]:
#Function that returns a pandas dataframe with the total number
#of months per year observations from a specific station regarding
#a specific data product are available for:
def get_nmonths_per_year(station_availability_df, time_start_col, time_end_col, st_column):
    
    def get_months_per_year_per_file(stationID, t_start, t_end):
        
        #Import modules:
        import pandas as pd
        from dateutil.relativedelta import relativedelta
        
        #Create empty list to store tuples of: "stationID", "year", "month"
        #[('GAT132', 2016, 5)]
        ls = []
        
        #Set current date to start date:
        cur_date = t_start
        
        #Loop through all months within start and end dates:
        while cur_date < t_end+relativedelta(hours=1):
            
            #Add tuple with "stationID", current date "year" and "month" to list:
            ls.append((stationID, cur_date.year, cur_date.month))
            
            #Change current date to one month ahead:
            cur_date += relativedelta(months=1)
            
        return pd.DataFrame(ls, columns=[st_column, 'year', 'month'])
        
        
    #Return pandas dataframe: 
    return pd.concat([get_months_per_year_per_file(station_availability_df[st_column].iloc[i],
                                                   station_availability_df[time_start_col].iloc[i],
                                                   station_availability_df[time_end_col].iloc[i])
                      for i in range(len(station_availability_df))]).drop_duplicates().reset_index().drop('index', axis=1)

In [ ]:
def data_availability_per_station_m(icos_df, st_col, sampl_height_col, data_prod_col, data_product, time_start_col, time_end_col):
    
    #Get a dataframe including how many data files are available per data product and station:
    availability_data_prod = icos_df.loc[icos_df[data_prod_col]==data_product].groupby([st_col,
                                                                                        sampl_height_col,
                                                                                        data_prod_col]).size()

    
    #Get list of dataframes with number of months per year observations
    #are available for per station:
    availability_nmomths_per_year_df = pd.concat([get_nmonths_per_year(icos_df[[st_col,
                                                                                data_prod_col,
                                                                                time_start_col,
                                                                                time_end_col]].loc[(icos_df[data_prod_col]==data_product) &
                                                                                                   (icos_df[st_col]==availability_data_prod.index[i][0])].reset_index(),
                                                                       time_start_col,
                                                                       time_end_col,
                                                                       st_col)
                                                  for i in range(len(availability_data_prod))]).reset_index().drop('index', axis=1)


    

    #Return dataframe:
    return availability_nmomths_per_year_df  

In [ ]:
def get_availability_per_month_per_station(stilt_path, stilt_icos_df, source, stilt_st_col, sampl_height_col, data_prod_col, data_product, time_start_col, time_end_col):
    
    #If input dataset consists of ICOS observations:
    if(source=='icos'):
    
        #Call function to get data availability for given ICOS data product: 
        prod_availability_df = data_availability_per_station_m(stilt_icos_df,
                                                               stilt_st_col,
                                                               sampl_height_col,
                                                               data_prod_col,
                                                               data_product,
                                                               time_start_col,
                                                               time_end_col)
        
    #If input dataset is STILT:
    else:
        
        #Call function to get data availability for STILT data: 
        prod_availability_df = get_stilt_availability_df(stilt_path,
                                                         stilt_icos_df,
                                                         stilt_st_col)
    #Add column with date:
    prod_availability_df['date']= pd.to_datetime(prod_availability_df.year*10000+prod_availability_df.month*100+1,format='%Y%m%d')

    #Add column with decimal date:
    prod_availability_df['decimalDate'] = [toYearFraction(prod_availability_df.date.iloc[i])
                                           for i in range(len(prod_availability_df))]
      
        
    #Return dataframe:
    return prod_availability_df

In [ ]:
def compute_sets1(icos_df, stilt_st_col, sampl_height_col, data_prod_col, data_product_ls, time_start_col, time_end_col):
    
    #Call function to get data availability for ICOS L1 CO2 atmospheric product: 
    icos_prod_availability_df = get_availability_per_month_per_station('',
                                                                      icos_df,
                                                                      'icos',
                                                                      stilt_st_col,
                                                                      sampl_height_col,
                                                                      data_prod_col,
                                                                      data_product_ls[0],
                                                                      time_start_col,
                                                                      time_end_col)


    
    
    #Return list of sets:
    return [icos_prod_availability_df]




In [ ]:
def compute_sets2(path_stilt, icos_df, stilt_st_col, sampl_height_col, data_prod_col, data_product_ls, time_start_col, time_end_col):
    
    #Call function to get data availability for ICOS L1 CO2 atmospheric product: 
    prod1_df = get_availability_per_month_per_station(path_stilt,
                                                      icos_df,
                                                      'icos',
                                                      stilt_st_col,
                                                      sampl_height_col,
                                                      data_prod_col,
                                                      data_product_ls[0],
                                                      time_start_col,
                                                      time_end_col)


    
    #Call function to get data availability for ICOS L1 CO2 atmospheric product: 
    prod2_df = get_availability_per_month_per_station(path_stilt,
                                                      icos_df,
                                                      'icos',
                                                      stilt_st_col,
                                                      sampl_height_col,
                                                      data_prod_col,
                                                      data_product_ls[1],
                                                      time_start_col,
                                                      time_end_col)

    

    #Merge availability-DFs for ICOS L1 & ICOS L2:
    icos_L1_L2_intersect_df = pd.merge(prod1_df,
                                       prod2_df,
                                       how='inner',
                                       on=list(prod2_df.columns.values), sort=True)
    
   

    #Set the combination of 'stiltStationId', 'date' as index:
    icos_L1_L2_intersect_df.set_index([stilt_st_col, 'date'], inplace=True)
    prod1_df.set_index([stilt_st_col, 'date'], inplace=True)
    prod2_df.set_index([stilt_st_col, 'date'], inplace=True)
    
    #ICOS L1 & ICOS L2:
    icos_L1_L2_df2 = icos_L1_L2_intersect_df.reset_index()

    #ICOS L1:
    icos_L1_df2 = prod1_df[(~prod1_df.isin(icos_L1_L2_intersect_df))].dropna().reset_index()
    #ICOS L2:
    icos_L2_df2 = prod2_df[(~prod2_df.isin(icos_L1_L2_intersect_df))].dropna().reset_index()

    #Return list of sets:
    return [icos_L1_df2, icos_L2_df2, icos_L1_L2_df2]

In [ ]:
def compute_sets3(path_stilt, stilt_icos_df, stilt_st_col, sampl_height_col, data_prod_col, data_product_ls, time_start_col, time_end_col):
    
    #Call function to get data availability for ICOS L1 CO2 atmospheric product: 
    icos_L1_availability_df2 = get_availability_per_month_per_station(path_stilt,
                                                                      stilt_icos_df,
                                                                      'icos',
                                                                      stilt_st_col,
                                                                      sampl_height_col,
                                                                      data_prod_col,
                                                                      data_product_ls[0],
                                                                      time_start_col,
                                                                      time_end_col)


    
    #Call function to get data availability for ICOS L1 CO2 atmospheric product: 
    icos_L2_availability_df2 = get_availability_per_month_per_station(path_stilt,
                                                                      stilt_icos_df,
                                                                      'icos',
                                                                      stilt_st_col,
                                                                      sampl_height_col,
                                                                      data_prod_col,
                                                                      data_product_ls[1],
                                                                      time_start_col,
                                                                      time_end_col)

    #Call function to get data availability for ICOS L1 CO2 atmospheric product: 
    stilt_months_per_year_df = get_availability_per_month_per_station(path_stilt,
                                                                      stilt_icos_df,
                                                                      'icos',
                                                                      stilt_st_col,
                                                                      sampl_height_col,
                                                                      data_prod_col,
                                                                      data_product_ls[2],
                                                                      time_start_col,
                                                                      time_end_col)


    #Merge availability-DFs for ICOS L1 & ICOS L2:
    icos_L1_L2_intersect_df = pd.merge(icos_L1_availability_df2,
                                       icos_L2_availability_df2,
                                       how='inner',
                                       on=list(icos_L2_availability_df2.columns.values), sort=True)
    
    #Merge availability-DFs for ICOS L1 & STILT:
    icos_L1_STILT_intersect_df = pd.merge(icos_L1_availability_df2,
                                          stilt_months_per_year_df,
                                          how='inner',
                                          on=list(icos_L1_availability_df2.columns.values), sort=True)

    #Merge availability-DFs for ICOS L2 & STILT:
    icos_L2_STILT_intersect_df = pd.merge(icos_L2_availability_df2,
                                          stilt_months_per_year_df,
                                          how='inner',
                                          on=list(icos_L2_availability_df2.columns.values), sort=True)
    
    
    #STILT, ICOS L1 & ICOS L2:
    stilt_icos_L1_L2_df2 = pd.merge(icos_L1_STILT_intersect_df,
                                    icos_L2_STILT_intersect_df,
                                    how='inner',
                                    on=list(icos_L2_STILT_intersect_df.columns.values), sort=True)

    #Set the combination of 'stiltStationId', 'date' as index:
    icos_L1_L2_intersect_df.set_index([stilt_st_col, 'date'], inplace=True)
    icos_L1_STILT_intersect_df.set_index([stilt_st_col, 'date'], inplace=True)
    icos_L2_STILT_intersect_df.set_index([stilt_st_col, 'date'], inplace=True)
    stilt_months_per_year_df.set_index([stilt_st_col, 'date'], inplace=True)
    icos_L1_availability_df2.set_index([stilt_st_col, 'date'], inplace=True)
    icos_L2_availability_df2.set_index([stilt_st_col, 'date'], inplace=True)
    
    #STILT & ICOS L1:
    stilt_icos_L1_df2 = icos_L1_STILT_intersect_df[~icos_L1_STILT_intersect_df.isin(icos_L1_L2_intersect_df)].dropna().reset_index()

    #STILT & ICOS L2:
    stilt_icos_L2_df2 = icos_L2_STILT_intersect_df[~icos_L2_STILT_intersect_df.isin(icos_L1_L2_intersect_df)].dropna().reset_index()

    #STILT:
    stilt_df2 = stilt_months_per_year_df[(~stilt_months_per_year_df.isin(icos_L1_STILT_intersect_df))&
                                         (~stilt_months_per_year_df.isin(icos_L2_STILT_intersect_df))].dropna().reset_index()
    
    #ICOS L1 & ICOS L2:
    icos_L1_L2_df2 = icos_L1_L2_intersect_df[~icos_L1_L2_intersect_df.isin(stilt_months_per_year_df)].dropna().reset_index()

    #ICOS L1:
    icos_L1_df2 = icos_L1_availability_df2[(~icos_L1_availability_df2.isin(icos_L1_STILT_intersect_df))&
                                           (~icos_L1_availability_df2.isin(icos_L2_STILT_intersect_df))&
                                           (~icos_L1_availability_df2.isin(icos_L1_L2_intersect_df))].dropna().reset_index()
    #ICOS L2:
    icos_L2_df2 = icos_L2_availability_df2[(~icos_L2_availability_df2.isin(icos_L1_STILT_intersect_df))&
                                           (~icos_L2_availability_df2.isin(icos_L2_STILT_intersect_df))&
                                           (~icos_L2_availability_df2.isin(icos_L1_L2_intersect_df))].dropna().reset_index()

    #Return list of sets:
    return [icos_L1_df2,
            icos_L2_df2,
            stilt_df2,
            icos_L1_L2_df2,
            stilt_icos_L1_df2,
            stilt_icos_L2_df2,
            stilt_icos_L1_L2_df2]

In [ ]:
def get_icos_df(domain, df):
    
    #Rename column name:
    df.rename(columns={"samplingHeight": "samplingheight"}, inplace=True)
    
    #Check if there are sampling height entries that are None and replace with '':
    df.loc[(df.samplingheight.isna()),'samplingheight']='' 
            
    #Convert data type of columns with time-info from string to datetime:
    df['timeStart'] = pd.to_datetime(df['timeStart']) 
    df['timeEnd'] = pd.to_datetime(df['timeEnd']) 
    
    #Drop column with dobj-info:
    df = df.drop('dobj', axis=1)
    
    if(domain=='atmosphere'):
        
        #Corrections for stations without registered sampling height in Drought 2018 Data Product:
        df.loc[(df.stationId=='CMN') & (df.specLabel=='Drought 2018 Atmospheric Product'), 'samplingheight']='8.0'
        df.loc[(df.stationId=='PAL') & (df.specLabel=='Drought 2018 Atmospheric Product'), 'samplingheight']='12.0'

        
    #Add column with station ID & sampling height (e.g. GAT060):
    df['stationId_long'] = df.apply (lambda row: st_str(row,'samplingheight','stationId'), axis=1)   
    
    #Reset index:
    return df.reset_index().drop('index', axis=1)
    
    

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='update_availability_table_func'></a>

## 6. Update availability plot function

In [ ]:
def update_availability_table(icos_df, st_col, sampl_height_col, data_prod_col, data_product_ls, time_start_col, time_end_col):
    
    #Import modules:
    from tools.visualization.availability import plot_attr, availability_plot

    #Create a new object of the class PlotAttr:
    avail_plot_attr = plot_attr.PlotAttr()

    #Provide list of tick-values for y-axis:
    avail_plot_attr.y_range=list(reversed(sorted((list(icos_df.reset_index()[st_col].unique())))))
    
    #Set plot dimentions:
    avail_plot_attr.width = 900
    avail_plot_attr.height = 1800
    
    #Define plot title:
    avail_plot_attr.title_text = 'ICOS data product availability table'

    #Set x- & y-axis labels:
    avail_plot_attr.xaxis_label = 'Time'
    avail_plot_attr.yaxis_label = 'Station'

    #Provide dataframe column name for values that will be plotted on x- & y-axis:
    avail_plot_attr.col_name_xaxis = 'decimalDate'
    avail_plot_attr.col_name_yaxis = st_col
    
    #Set legend orientation:
    if(len(data_product_ls)>1):
        avail_plot_attr.legend_orientation = "vertical"
    else:
        avail_plot_attr.legend_orientation = "horizontal"
    
    #Get a list of dataframes with availability info for
    #every combination of ICOS L1, L2 & STILT data products
    #per station, year and month:
    if (len(data_product_ls)==1):
        data_set_ls = compute_sets1(icos_df,
                                   st_col,
                                   sampl_height_col,
                                   data_prod_col,
                                   data_product_ls,
                                   time_start_col,
                                   time_end_col)
    elif (len(data_product_ls)==2):
        data_set_ls = compute_sets2('',
                                    icos_df,
                                   st_col,
                                   sampl_height_col,
                                   data_prod_col,
                                   data_product_ls,
                                   time_start_col,
                                   time_end_col)
        
    elif (len(data_product_ls)==3):
        data_set_ls = compute_sets3('',
                                   icos_df,
                                   st_col,
                                   sampl_height_col,
                                   data_prod_col,
                                   data_product_ls,
                                   time_start_col,
                                   time_end_col)
    else:
        print('Invalid list of data products!')
    
    #Create a list with names for every layer:
    if(len(data_product_ls)==1):
        
        layer_ls = data_product_ls
    
    elif(len(data_product_ls)==2):
        
        layer_ls = [data_product_ls[0], data_product_ls[1],
                    data_product_ls[0]+' and '+data_product_ls[1]]
        
    elif(len(data_product_ls)==3):
        
        layer_ls = [data_product_ls[0], data_product_ls[1], data_product_ls[2],
                    data_product_ls[0]+' and '+data_product_ls[1],
                    data_product_ls[0]+' and '+data_product_ls[2],
                    data_product_ls[1]+' and '+data_product_ls[2],
                    data_product_ls[0]+', '+data_product_ls[1]+' and '+data_product_ls[2]]
    else:
        print('Invalid list of data products!')
        
    #layer_ls = ['STILT & ICOS L2', 'STILT & ICOS L1',
                #'ICOS L1', 'ICOS L1 & ICOS L2', 'ICOS L2',
                #'STILT, ICOS L1 & L2', 'STILT']
    
    #List of tuples for tooltips:
    t_list = [("Station", st_col),
              ("Year", "year"),
              ("Month", "month")]
    
    #Call function to display plot:
    availability_plot.plot_table(data_set_ls, layer_ls, t_list, avail_plot_attr)

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>

<a id='widget_func'></a>

## 7. Widget function

In [ ]:
def as_availability_wform():
    
    #Import modules:
    from ipywidgets import SelectMultiple, Dropdown, interact_manual
    import pandas as pd

##################################################################################################################
    #Function that returns a sorted list of data obj specs with their
    #corresponding URLs for a given domain:
    def dobj_spec_labels(domain_str):

        if((isinstance(domain_str, str)) & ((domain_str=='atmosphere') |
                                            (domain_str=='ecosystem') |
                                            (domain_str=='ocean'))):

            #Get an pandas df of data obj specs with URLs for the given domain:
            dobj_spec_array = RunSparql(sparqls.icos_prods_per_domain(domain_str),
                                        output_format='pandas').run().to_records(index=False)

            #Create a sorted list of tuples with dobj spec labels & their corresponding URLs:
            dobj_spec_label_ls = sorted([(dobj_spec[0], [dobj_spec[0], dobj_spec[1]]) for dobj_spec in dobj_spec_array])

        else:

            dobj_spec_label_ls = []

        #Return sorted list of tuples with dobj spec labels and
        #their corresponding URLs:
        return dobj_spec_label_ls

    
    
##################################################################################################################    
    def on_change_domain_widget(val):

        #Call function to get list of data products for selected domain:
        if (val.new=='atmosphere'):
            
            prods_wdgt.options = dobj_spec_labels('atmosphere')
                                                              
        elif (val.new=='ecosystem'):
            
            prods_wdgt.options = dobj_spec_labels('ecosystem')
            
        elif (val.new=='ocean'):
            
            prods_wdgt.options = dobj_spec_labels('ocean')
        else:
            print('Invalid domain...')     
                   
##########################################################################################

    #Set style of widget-description:
    style = {'description_width': '200px'}

    #Add dropdown widget with ICOS domains:
    domain_wdgt = Dropdown(options = ['atmosphere', 'ecosystem', 'ocean'],
                           description='Domain',
                           value='atmosphere')
    
    #Add event-handler (products):
    domain_wdgt.observe(on_change_domain_widget, 'value')
    
    #Add multi-select dropdown widget with ICOS data products for selected domain:
    prods_wdgt = SelectMultiple(options=dobj_spec_labels('atmosphere'),
                                #value=[''],
                                #rows=10,
                                description='Data products',
                                style= style,
                                disabled=False)
   
    
    
    #Function that updates the content of the availability
    #table, based on the user's selections:
    def update_func(domain, data_prods): 
        
        #Check how many data products the user has selected:
        if(len(data_prods)>3):
            
            print('Exceeded permitted number of selected data products!\nPlease select up to max 3 data products.')
        
        elif(len(data_prods)<1):
            
            print('No data product selected!\nPlease select a data product.')
        
        
        #The user can only select up to max 3 data products:
        else:
        
            #Convert tuple of lists to list of strings of data obj spec URLs:
            data_prods_url_ls = [s[1] for s in data_prods]
            
            #Get list of data obj spec:
            data_prods_ls = [s[0] for s in data_prods]            
            
            #Get metadata about all stations that produce the selected data products:
            # icos_df = get_icos_df(
               # domain,
               # RunSparql(sparqls.prod_availability(data_prods_url_ls),output_format='pandas').run())

            # Temporary fix to filter out incomplete uploads.
            icos_df = get_icos_df(
                domain,
                RunSparql(fixed_prod_availability(data_prods_url_ls),output_format='pandas').run())
            # End of temporary fix.

            #Call function to update content of availability table:
            update_availability_table(icos_df,
                                      'stationId_long',
                                      'samplingheight',
                                      'specLabel',
                                      data_prods_ls,
                                      'timeStart',
                                      'timeEnd')






    #Create SelectMultiple dropdown widget with
    #ICOS atmospheric data products:
    interact_m = interact_manual(update_func,
                                 domain = domain_wdgt,
                                 data_prods = prods_wdgt)

    #Format widgets:
    interact_m.widget.children[0].layout.width = '410px'
    interact_m.widget.children[0].layout.margin = '40px 2px 2px 213px'
    interact_m.widget.children[1].layout.width = '530px'
    interact_m.widget.children[1].layout.height = '135px'
    interact_m.widget.children[1].layout.margin = '10px 2px 2px 92px'
    interact_m.widget.children[2].description = 'Update Table'
    interact_m.widget.children[2].button_style = 'danger'
    interact_m.widget.children[2].style.button_color = '#3973ac'
    interact_m.widget.children[2].layout.margin = '10px 10px 40px 390px' # top/right/bottom/left

    def fixed_prod_availability(dobj_spec_ls):
        # Code copied directly from the pylib repository.
        # The query is revisited according to Oleg's suggestions.
        # Check input:
        if (isinstance(dobj_spec_ls, list) & (len(dobj_spec_ls)>0) & (len(dobj_spec_ls)<4)):
            #Add '<' and '>' at the begining and the end of every dobj spec label:
            dobj_spec_ls_frmt = ['<'+i+'>' for i in dobj_spec_ls if isinstance(i, str)]
            #Export list items to a string:
            dobj_spec_labels = ' '.join(dobj_spec_ls_frmt)
            query = """prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
            prefix cpmeta: <http://meta.icos-cp.eu/ontologies/cpmeta/>
            prefix prov: <http://www.w3.org/ns/prov#>
            select ?dobj ?samplingHeight ?specLabel ?timeStart ?timeEnd ?stationId
            where {
                {
                    select * where {
                        VALUES ?spec {"""+dobj_spec_labels+"""
                        }
                        ?dobj cpmeta:hasObjectSpec ?spec .
                        ?dobj cpmeta:wasAcquiredBy [
                            prov:startedAtTime ?timeStart ;
                            prov:endedAtTime ?timeEnd ;
                            prov:wasAssociatedWith ?station
                        ] .
                        optional {?dobj cpmeta:wasAcquiredBy/cpmeta:hasSamplingHeight ?samplingHeight }
                        FILTER NOT EXISTS {[] cpmeta:isNextVersionOf ?dobj}
                        FILTER EXISTS {?dobj cpmeta:hasSizeInBytes []}
                    }
                }
                ?station cpmeta:hasStationId ?stationId .
                ?spec rdfs:label ?specLabel
            }"""
        else:
            # Prompt error message:
            print('Invalid entry! The input has to be a list of 1 to max 3 items.')

            # Empty query:
            query = ''
        # Return query-string:
        return query

<br>
<div style="text-align: right"> 
    <a href="#intro">Back to top</a>
</div>
<br>
<br>
<br>
